# Pandas (Parte 3)

In [2]:
import pandas as pd
import numpy as np


## Visualizando los tipos de merge

<img src="img/joinimages.png" width="500">

- Inner join: how = "inner"
- Left outer join : how = "left"
- Right outer join: how = "right"
- Full outer join: how = "outer"

Tomado de https://www.geeksforgeeks.org/python-pandas-merging-joining-and-concatenating/


In [ ]:
locacion_datos = "https://otorongo.club/2021/json/ingresos/"

cong = pd.read_json(locacion_datos)

In [ ]:
cong_1 = cong[['dni', 'total_ingreso']]

cong_2 = cong[['dni', 'partido']]

solo_vn = cong_2.loc[cong_2['partido'] == 'VICTORIA NACIONAL', :]

Cómo es un outer merge? 

In [ ]:
resultado_merge_vn = pd.merge(cong_1,
                     solo_vn,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'outer')

resultado_merge_vn
#resultado_merge_vn.loc[resultado_merge_vn['partido'] != 'VICTORIA NACIONAL']
#resultado_merge_vn.loc[resultado_merge_vn['partido'] == 'VICTORIA NACIONAL']

Cómo es un left merge? Y un right merge? 


In [ ]:
resultado_merge_vn = pd.merge(cong_1,
                     solo_vn,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'left')
#                    how = 'right')

resultado_merge_vn

### Uniendo varias bases de datos verticalmente (append)

En pandas, este método se llama concat. 

In [ ]:
df_partidos = {}
for cat in cong['partido'].unique():
    df = cong.loc[cong['partido'] == cat]
    df_partidos[cat] = df

In [ ]:
df_fp = df_partidos['FUERZA POPULAR']
df_vn = df_partidos['VICTORIA NACIONAL']

In [ ]:
df_append = pd.concat([df_vn,df_fp])

In [ ]:
#df_append

In [ ]:
df_append = pd.concat([df_partidos['FUERZA POPULAR'], df_partidos['VICTORIA NACIONAL']])

### Limpieza de datos (parte 2)

Eliminando columnas

In [ ]:
#Creando (y eliminando) una columna que refleja un mal cálculo
cong['calculo_mal_hecho'] = cong.eval('ingreso_publico + ingreso_privado')

In [ ]:
cong.drop(columns = 'calculo_mal_hecho', inplace = True)


Identificando (y eliminando) duplicados:

In [ ]:
nueva_df = pd.concat([cong, cong])

In [ ]:
nueva_df.head(5)

In [ ]:
nueva_df.nunique()

In [ ]:
nueva_df.nunique(dropna=False)

In [ ]:
## Cuales son las dimensiones de nuestra dataframe si eliminamos las observaciones duplicadas
nueva_df.drop_duplicates().shape

In [ ]:
nueva_df.shape

In [ ]:
sin_duplicados = nueva_df.drop_duplicates(subset='dni')
sin_duplicados.head(5)

In [ ]:
sin_duplicados_2 = nueva_df.drop_duplicates(subset=['dni','partido'])


### Manejando valores perdidos (missing)

A continuación veremos cómo manejar valores perdidos (missing). Veremos 3 métodos en especial:
- ` isna`
- `dropna`
- `fillna`

In [ ]:
cong_m = cong.copy(deep = True)

In [ ]:
cong_m.replace({0:np.nan}, inplace = True)

In [ ]:
cong_m.isna().head(5)

In [ ]:
cong_m.isna().sum()

In [ ]:
cong_m.isna().mean().round(4)*100

In [ ]:
## Aquí vemos cuales observaciones se eliminarían 
cong_m.dropna().shape

In [ ]:
cong_m.dropna(subset=['ingreso_publico', 'ingreso_privado']).shape

In [ ]:
#cong_m.fillna('hola')

In [ ]:
dict_fill = {'total_ingreso': 0, 'ingreso_publico': 'hola'}
#cong_m.fillna(dict_fill)

In [ ]:
med_fill = cong_m.median(numeric_only=True)
med_fill

In [ ]:
med_fill = cong_m.median(numeric_only=True)

cong_m.fillna(med_fill)

### Haciendo un reshape de los datos (volver los datos de wide a long, y viceversa).

<img src="img/wlong.png" width="500">
de: https://www.statology.org/long-vs-wide-data/

In [ ]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 10000, 50000, 100000, 200000000]
cong['cat_ingreso'] = pd.cut(cong['total_ingreso'],
                              bins = categ_bins, labels = categ_labels)



In [ ]:
cong.head(3)

In [ ]:
resumen = cong.pivot_table(index='partido', columns='cat_ingreso', values='total_ingreso', aggfunc='mean')

In [ ]:
resumen.columns = resumen.columns.astype(str) ### Tengo que convertir mis columnas, que eran "Categorical indexes" en strings (no siempre pasara esto, suele estar en string )
#resumen['total'] =resumen.sum(axis=1)

In [ ]:
resumen = resumen.reset_index()

In [ ]:
resumen_melted = resumen.melt(id_vars = 'partido', value_vars = ['cat_1', 'cat_2', 'cat_3', 'cat_4'])
resumen_melted

## Apply
A veces queremos aplicarle una función a cada observación de una variable. Esto no se puede directamente ya que pandas hace cálculos vectoriales. En estos casos usamos el apply. 


In [ ]:
## Usando las mismas categorías de ingresos de antes:
def clasif_ing(ing):
    if ing < 10000:
        new_ing = 'cat_1'
    elif ((ing >= 10000) & (ing < 50000)):
          new_ing = 'cat_2'
    else:
          new_ing = 'cat_3'
    return new_ing


In [ ]:
cong['nueva_cat'] = cong['total_ingreso'].apply(lambda x: clasif_ing(x))
cong[['total_ingreso', 'nueva_cat']]